In [1]:
!pip install gensim==3.8.1
from gensim.models import KeyedVectors

path_model = "embeddings-l-model.vec"
# Load vectors directly from the file
modelWV = KeyedVectors.load_word2vec_format(path_model, limit=300000)
DIMENSION=300

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from datasets import load_dataset, DatasetDict

raw_dataset = load_dataset("javilonso/mex_data_augmented", use_auth_token=True)

Downloading:   0%|          | 0.00/961 [00:00<?, ?B/s]

Using custom data configuration javilonso--mex_data_augmented-869fe1499d7b863d


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /home/javilonso/.cache/huggingface/datasets/parquet/javilonso--mex_data_augmented-869fe1499d7b863d/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Extract data
X_train = raw_dataset['train']['Opinion']
y_train_attraction = raw_dataset['train']['Attraction']
y_train_polarity = raw_dataset['train']['Polarity']

X_test = raw_dataset['test']['Opinion']
y_test_attraction = raw_dataset['test']['Attraction']
y_test_polarity = raw_dataset['test']['Polarity']


In [4]:
import numpy as np

def getDocVec(text):
    """gets a text and returns a vector for this text"""

    tokens=text.split()
    data=[]
    n=0
    for token in tokens:
        token=token.lower()
        if token in modelWV.vocab:
            data.append(modelWV[token])
            n += 1
            
    if len(data)>0:     
        #vectors = np.array(data)
        res = [sum(i)/n for i in zip(*data)]
    else:
        res =[0]*DIMENSION

    #return np.average(vectors, axis=0)
    return res

# text="Esto es un ejemplo de texto"
# print(len(getDocVec(text)))

In [5]:
X_vectors_train = []
for text in X_train:
  docVector = getDocVec(text)
  X_vectors_train.append(docVector)
  
print('text vectors for training were created' )

text vectors for training were created


In [6]:
X_vectors_test = []
for text in X_test:
  docVector=getDocVec(text)
  X_vectors_test.append(docVector)
  
print('text vectors for test were created' )

text vectors for test were created


In [7]:
print(len(X_vectors_train), len(y_train_attraction))
print(len(X_vectors_train[0]))
print(len(X_vectors_test), len(y_test_attraction))


30840 30840
300
7711 7711


In [8]:
from sklearn import svm

modelSVM=svm.SVC()
modelSVM.fit(X_vectors_train, y_train_attraction)
print('model was trained!!')

model was trained!!


In [9]:
# TEST Predictions
from sklearn.metrics import classification_report

y_pred=modelSVM.predict(X_vectors_test)
print(classification_report(y_test_attraction, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3906
           1       0.92      0.90      0.91      2337
           2       0.95      0.94      0.95      1468

    accuracy                           0.94      7711
   macro avg       0.94      0.94      0.94      7711
weighted avg       0.94      0.94      0.94      7711



In [10]:
from sklearn import svm

modelSVM=svm.SVC()
modelSVM.fit(X_vectors_train,y_train_polarity)
print('model was trained!!')

model was trained!!


In [11]:
# TEST Predictions
from sklearn.metrics import classification_report

y_pred=modelSVM.predict(X_vectors_test)
print(classification_report(y_test_polarity, y_pred))


              precision    recall  f1-score   support

           0       0.56      0.33      0.42       481
           1       0.64      0.01      0.02       610
           2       0.52      0.70      0.60      1290
           3       0.00      0.00      0.00      1113
           4       0.73      0.99      0.84      4217

    accuracy                           0.68      7711
   macro avg       0.49      0.40      0.37      7711
weighted avg       0.57      0.68      0.59      7711



/home/javilonso/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/javilonso/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/javilonso/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
